In [1]:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 - using this model for sentence-bert. This has semantic understanding.

In [4]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

e:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
df = pd.read_csv('synthetic_names_samples.csv')

In [7]:
model.encode(df['transcription'])

e:\Anaconda\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


array([[-0.01437728, -0.00956397,  0.00772832, ..., -0.05515455,
        -0.0196485 ,  0.04890266],
       [-0.0160002 , -0.06145441,  0.00687232, ..., -0.03596389,
         0.01290313,  0.03383428],
       [ 0.04957252,  0.0404234 , -0.03041152, ..., -0.03086681,
        -0.02787871,  0.05693854],
       ...,
       [-0.03077874, -0.00136129, -0.07155177, ..., -0.03800274,
        -0.01676858,  0.05663359],
       [-0.01091649,  0.09627073, -0.02945522, ..., -0.02109083,
         0.05302236,  0.06531279],
       [-0.04220346,  0.02071806,  0.0709947 , ..., -0.03670705,
        -0.04892308,  0.07799632]], dtype=float32)